<a href="https://colab.research.google.com/github/deiveleal/data/blob/main/mestrado/ft105/classificacao/ClassificationNaiveBayesKNNEnsemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FT105A - Tópico Interdisciplinar I: Introdução ao Aprendizado de Máquina
### Aluno: Deive Audieres Leal
### RA: 083423


### Tarefa 2: Naïve Bayes, k-NN e Ensembles

#### Link para o repositório: [ClassificationNaiveBayesKNNEnsemble](https://github.com/deiveleal/data/blob/main/mestrado/ft105/classificacao/ClassificationNaiveBayesKNNEnsemble.ipynb)

# Enunciado:

Escolha um pacote de software para classificação de dados que tenha implementações de algoritmos de árvores de decisão, Naïve Bayes e k-NN, e aplique estes três algoritmos ao conjunto de dados Liver Disorder:

* Utilize subamostragem aleatória com 5 repetições para cada algoritmo e apresente o erro de classificação médio de cada um (para os conjuntos de testes);
* Adote uma divisão de 70% dos dados para treinamento e 30% dos dados para teste;
* Faça a amostragem antes de iniciar o treinamento e use os mesmos dados para todos os algoritmos (em cada repetição);

Para cada repetição, monte um ensemble com os classificadores já treinados (via voto majoritário), aplique ao conjunto de testes e apresente o desempenho médio.

ATENÇÃO: Não se esqueça de apresentar no relatório os parâmetros definidos para cada algoritmo (caso existam)!

ATENÇÃO: Verifique na documentação do conjunto de dados como definir o atributo alvo, e explique o procedimento adotado no relatório.

#### Instala o repositório da UCI

In [1]:
!pip install ucimlrepo

#### Importação das bibliotecas

In [2]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import naive_bayes as nb
from sklearn import neighbors
from ucimlrepo import fetch_ucirepo

#### Importa o conjunto de dados

#### Realiza a busca dos dados


In [3]:
liver_disorders = fetch_ucirepo(id=60)

#### Os dados são um dataframe pandas já separado em features e targets

In [31]:
X = liver_disorders.data.features.astype(str)
y = liver_disorders.data.targets.astype(str)

In [32]:
# metadata
print("Metadata:\n")
liver_disorders.metadata

Metadata:



{'uci_id': 60,
 'name': 'Liver Disorders',
 'repository_url': 'https://archive.ics.uci.edu/dataset/60/liver+disorders',
 'data_url': 'https://archive.ics.uci.edu/static/public/60/data.csv',
 'abstract': 'BUPA Medical Research Ltd. database donated by Richard S. Forsyth',
 'area': 'Health and Medicine',
 'tasks': ['Regression'],
 'characteristics': ['Multivariate'],
 'num_instances': 345,
 'num_features': 5,
 'feature_types': ['Categorical', 'Integer', 'Real'],
 'demographics': [],
 'target_col': ['drinks'],
 'index_col': None,
 'has_missing_values': 'no',
 'missing_values_symbol': None,
 'year_of_dataset_creation': 2016,
 'last_updated': 'Fri Nov 03 2023',
 'dataset_doi': '10.24432/C54G67',
 'creators': [],
 'intro_paper': None,
 'additional_info': {'summary': 'The first 5 variables are all blood tests which are thought to be sensitive to liver disorders that might arise from excessive alcohol consumption. Each line in the dataset constitutes the record of a single male individual.\n\n

In [33]:
# variable information
print("Variables: \n", liver_disorders.variables)

Variables: 
        name     role         type demographic  \
0       mcv  Feature   Continuous        None   
1   alkphos  Feature   Continuous        None   
2      sgpt  Feature   Continuous        None   
3      sgot  Feature   Continuous        None   
4   gammagt  Feature   Continuous        None   
5    drinks   Target   Continuous        None   
6  selector    Other  Categorical        None   

                                         description units missing_values  
0                            mean corpuscular volume  None             no  
1                               alkaline phosphotase  None             no  
2                           alanine aminotransferase  None             no  
3                         aspartate aminotransferase  None             no  
4                      gamma-glutamyl transpeptidase  None             no  
5  number of half-pint equivalents of alcoholic b...  None             no  
6  field created by the BUPA researchers to split...  None    

### Mostra o dataframe montado

In [34]:
df = pd.concat([X,y], axis=1)
df.head(7)

,mcv,alkphos,sgpt,sgot,gammagt,drinks
0,85,92,45,27,31,0.0
1,85,64,59,32,23,0.0
2,86,54,33,16,54,0.0
3,91,78,34,24,36,0.0
4,87,70,12,28,10,0.0
5,98,55,13,17,17,0.0
6,88,62,20,17,9,0.5


## Classificação com Árvore de Decisão

#### Primeira execução

##### Separa o dataset em 80% de treino e 20% de teste, de forma randomica e com embaralhamento, foi utilizado o valor 52 no random_state como semente, para que ao alterar o mesmo na próxima execução, os valores escolhidos para treino e teste sejam diferentes

In [42]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3
    )

In [43]:
X_train

,mcv,alkphos,sgpt,sgot,gammagt
240,85,62,15,13,22
20,92,59,35,13,19
152,90,63,12,26,21
272,91,71,12,22,11
191,88,56,23,18,12
...,...,...,...,...,...
94,89,62,42,30,20
176,91,69,25,25,66
158,93,99,36,34,48
146,96,70,70,26,36


##### Cria função com o modelo de árvore de decisão. Foram usados os parâmetros padrões com exceção do criterio que passei a utilizar o 'log_loss' como ganho da informação ao invés do padrão gini. É retornado a acurácia do modelo ao final da execução.

In [50]:
def tree_classifier(X_train, X_test, y_train, y_test):
    clf = tree.DecisionTreeClassifier(criterion='log_loss')
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return accuracy

##### Acurácia do modelo! O quanto ele acertou?

In [53]:
print("Acurácia Tree:", tree_classifier(
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test
    ))

Acurácia Tree: 0.15384615384615385


In [59]:
def knn_classifier(X_train, X_test, y_train, y_test):
    clf = neighbors.KNeighborsClassifier()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return accuracy

In [58]:
print("Acurácia KNN:", knn_classifier(
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test
    ))

Acurácia KNN: 0.2692307692307692


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [60]:
def nb_classifier(X_train, X_test, y_train, y_test):
    clf = nb.GaussianNB()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return accuracy

In [61]:
print("Acurácia NB:", nb_classifier(
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test
    ))

Acurácia NB: 0.2980769230769231


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [76]:
tree_accuracy = []
knn_accuracy = []
nb_accuracy = []
for execution in range(0, 25):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    tree_accuracy.append(tree_classifier(X_train, X_test, y_train, y_test))
    knn_accuracy.append(tree_classifier(X_train, X_test, y_train, y_test))
    nb_accuracy.append(tree_classifier(X_train, X_test, y_train, y_test))

In [77]:
df_ensemble = pd.DataFrame()

In [78]:
df_ensemble['tree_accuracy'] = tree_accuracy
df_ensemble['knn_accuracy'] = knn_accuracy
df_ensemble['nb_accuracy'] = nb_accuracy

In [79]:
df_ensemble

,tree_accuracy,knn_accuracy,nb_accuracy
0,0.173077,0.125000,0.134615
1,0.163462,0.173077,0.192308
2,0.144231,0.134615,0.115385
3,0.192308,0.201923,0.221154
4,0.144231,0.144231,0.173077
5,0.182692,0.201923,0.201923
6,0.192308,0.211538,0.192308
7,0.221154,0.211538,0.221154
8,0.182692,0.173077,0.201923
9,0.173077,0.192308,0.173077


In [80]:
df_ensemble.max().mean()

0.23076923076923075